In [1]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense  
from keras import backend as K
import trackml

In [2]:
# training dataset - one event
from trackml.dataset import load_event

hits, cells, particles, truth = load_event('../TrackML-Data/train_100_events/event000001000')

hits

,hit_id,x,y,z,volume_id,layer_id,module_id
0,1,-64.409897,-7.163700,-1502.5,7,2,1
1,2,-55.336102,0.635342,-1502.5,7,2,1
2,3,-83.830498,-1.143010,-1502.5,7,2,1
3,4,-96.109100,-8.241030,-1502.5,7,2,1
4,5,-62.673599,-9.371200,-1502.5,7,2,1
...,...,...,...,...,...,...,...
120934,120935,-763.862976,51.569401,2944.5,18,12,97
120935,120936,-808.705017,3.459260,2944.5,18,12,97
120936,120937,-982.935974,41.460899,2952.5,18,12,98
120937,120938,-942.698975,18.489100,2952.5,18,12,98


In [7]:
# number of hit pairs for event
print(((hits.shape[0])**2-hits.shape[0])/2)

7313060391.0


In [3]:
cells

,hit_id,ch0,ch1,value
0,1,209,617,0.013832
1,1,210,617,0.079887
2,1,209,618,0.211723
3,2,68,446,0.334087
4,3,58,954,0.034005
...,...,...,...,...
664991,120936,122,5,1.000000
664992,120936,123,5,1.000000
664993,120937,430,11,1.000000
664994,120938,260,7,1.000000


In [4]:
for i in range(30):
    print(cells['hit_id'][i], cells['value'][i])
print(cells.shape)

1 0.0138317
1 0.0798866
1 0.211723
2 0.334087
3 0.0340049
3 0.00779792
3 0.0198969
3 0.0999644
3 0.0655761
3 0.0186967
3 0.0941926
3 0.0633718
4 0.323907
5 0.296566
6 0.289269
7 0.304021
8 0.287819
9 0.329157
10 0.236652
10 0.00364747
11 0.282262
12 0.0562005
12 0.141562
12 0.110228
13 0.28905
14 0.057594
14 0.238345
15 0.301006
16 0.346271
17 0.26638
(664996, 4)


In [5]:
# detector information
detectors = pd.read_csv("../TrackML-Data/detectors.csv")
detectors

,volume_id,layer_id,module_id,cx,cy,cz,rot_xu,rot_xv,rot_xw,rot_yu,...,rot_yw,rot_zu,rot_zv,rot_zw,module_t,module_minhu,module_maxhu,module_hv,pitch_u,pitch_v
0,7,2,1,-65.7965,-5.17830,-1502.5,0.078459,-0.996917,0.0,-0.996917,...,0.0,0,0,-1,0.15,8.4,8.4,36,0.05,0.05625
1,7,2,2,-139.8510,-6.46568,-1502.0,0.046184,-0.998933,0.0,-0.998933,...,0.0,0,0,-1,0.15,8.4,8.4,36,0.05,0.05625
2,7,2,3,-138.6570,-19.34190,-1498.0,0.138156,-0.990410,0.0,-0.990410,...,0.0,0,0,-1,0.15,8.4,8.4,36,0.05,0.05625
3,7,2,4,-64.1764,-15.40740,-1498.0,0.233445,-0.972370,0.0,-0.972370,...,0.0,0,0,-1,0.15,8.4,8.4,36,0.05,0.05625
4,7,2,5,-136.2810,-32.05310,-1502.0,0.228951,-0.973438,0.0,-0.973438,...,0.0,0,0,-1,0.15,8.4,8.4,36,0.05,0.05625
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18723,18,12,94,-895.8950,291.09400,2952.5,0.309017,-0.951057,0.0,0.951057,...,0.0,0,0,1,0.35,66.0,72.0,78,0.12,10.40000
18724,18,12,95,-812.0900,161.53500,2947.5,0.195090,-0.980785,0.0,0.980785,...,0.0,0,0,1,0.35,54.0,64.2,78,0.12,10.40000
18725,18,12,96,-925.3150,176.51300,2955.5,0.187381,-0.982287,0.0,0.982287,...,0.0,0,0,1,0.35,66.0,72.0,78,0.12,10.40000
18726,18,12,97,-826.2270,54.15380,2944.5,0.065403,-0.997859,0.0,0.997859,...,0.0,0,0,1,0.35,54.0,64.2,78,0.12,10.40000


Choice of inputs for each hit
- x,y,z
- sum of values from cells, count of number of cells, normalized important?
- from volume/layer/module id's: cx,cy,cz, ...? (leaving for now)

In [20]:
# dataset for one event
current_id = cells['hit_id'][0]
cells_count = 1
value_count = cells['value'][row]
cells_list = []
value_list = []

for row in range(1, len(cells['hit_id'])):
    hit_id = cells['hit_id'][row]
    hit_value = cells['value'][row]
    if (hit_id != current_id):
        cells_list.append(cells_count)
        value_list.append(value_count)
        current_id = hit_id
        cells_count = 1
        value_count = hit_value
    else:
        cells_count += 1
        value_count += hit_value
        
cells_arr = np.array(cells_list)
value_arr = np.array(value_list)
print(cells_arr.shape)

(120938,)
